In [121]:
#import dependencies
from __future__ import division
import numpy as np
import pandas as pd
import re
import h5py
import unidecode
from tqdm import tqdm
import matplotlib
% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Conv1D, Dropout, Flatten, BatchNormalization
from keras.models import Model
from keras import metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from keras import backend as K
from scipy.sparse import vstack

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
levels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

#summing the toxicity levels so that we can easily divide the train data to K folds.
train['sum_level'] = train[levels[0]] + train[levels[1]] + train[levels[2]] + train[levels[3]] + train[levels[4]] + train[levels[5]]

In [ ]:
#we see there are many \n characters in text. lets just remove those first
good_text = []
for i in tqdm(train['comment_text']):
    i = re.sub(r'[\n]+', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    good_text.append(i)
train['comment_text'] = good_text


#we see there are many \n characters in text. lets just remove those first
good_text_test = []
for i in tqdm(test['comment_text']):
    i = re.sub(r'[\n]+', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    good_text_test.append(i)
test['comment_text'] = good_text_test


#let us strip the unicode accents

good_text = []
for i in tqdm(train['comment_text']):
    i = unicode(i, 'utf-8')
    i = unidecode.unidecode(i)
    good_text.append(i)
train['comment_text'] = good_text


good_text_test = []
for i in tqdm(test['comment_text']):
    i = unicode(i, 'utf-8')
    i = unidecode.unidecode(i)
    good_text_test.append(i)
test['comment_text'] = good_text_test


good_text = []
for i in tqdm(train['comment_text']):
    i = i.lower()
    i = re.sub(r'\\\'s', ' is', i)
    i = re.sub(r'\'s', ' is', i)
    
    i = re.sub(r'can\\\'t', 'can not', i)
    i = re.sub(r'can\'t', 'can not', i)
    
    i = re.sub(r'n\\\'t', ' not', i)
    i = re.sub(r'n\'t', ' not', i)
    
    i = re.sub(r'\\\'nt', ' not', i)
    i = re.sub(r'\'nt', ' not', i)
    
    i = re.sub(r'\\\'re', ' are', i)
    i = re.sub(r'\'re', ' are', i)
    
    i = re.sub(r'\s[w]\'d', ' would', i)
    i = re.sub(r'\\\'d', ' would', i)
    i = re.sub(r'\'d', ' would', i)
    
    i = re.sub(r'\\\'ll', ' will', i)
    i = re.sub(r'\'ll', ' will', i)
    
    i = re.sub(r'i\\\'m', ' i am ', i)
    i = re.sub(r'i\'m', ' i am ', i)
    
    i = re.sub(r'\\\'pedia', ' wikipedia ', i)
    i = re.sub(r'\'pedia', ' wikipedia ', i)
    
    i = re.sub(r'https://www\.', ' www ', i)
    i = re.sub(r'www\.', ' www ', i)
    
    i = re.sub(r'\.com', ' com ', i)
    
    i = re.sub(r'[-]+', ' ', i)
    
    i = re.sub(r'[\[ \] \. " # \$ % \^ \* \( \) \? \\ / @ < > _ : = \+ \{ } \| ~ ! , \']+', ' ', i)
    
    i = re.sub(r'\s+', ' ', i)
    
    i = i.strip()
    
    good_text.append(i)
train['comment_text'] = good_text




good_text_test= []
for i in tqdm(test['comment_text']):
    i = i.lower()
    i = re.sub(r'\\\'s', ' is', i)
    i = re.sub(r'\'s', ' is', i)
    
    i = re.sub(r'can\\\'t', 'can not', i)
    i = re.sub(r'can\'t', 'can not', i)
    
    i = re.sub(r'n\\\'t', ' not', i)
    i = re.sub(r'n\'t', ' not', i)
    
    i = re.sub(r'\\\'nt', ' not', i)
    i = re.sub(r'\'nt', ' not', i)
    
    i = re.sub(r'\\\'re', ' are', i)
    i = re.sub(r'\'re', ' are', i)
    
    i = re.sub(r'\s[w]\'d', ' would', i)
    i = re.sub(r'\\\'d', ' would', i)
    i = re.sub(r'\'d', ' would', i)
    
    i = re.sub(r'\\\'ll', ' will', i)
    i = re.sub(r'\'ll', ' will', i)
    
    i = re.sub(r'i\\\'m', ' i am ', i)
    i = re.sub(r'i\'m', ' i am ', i)
    
    i = re.sub(r'\\\'pedia', ' wikipedia ', i)
    i = re.sub(r'\'pedia', ' wikipedia ', i)
    
    i = re.sub(r'https://www\.', ' www ', i)
    i = re.sub(r'www\.', ' www ', i)
    
    i = re.sub(r'\.com', ' com ', i)
    
    i = re.sub(r'[-]+', ' ', i)
    
    i = re.sub(r'[\[ \] \. " # \$ % \^ \* \( \) \? \\ / @ < > _ : = \+ \{ } \| ~ ! , \']+', ' ', i)
    
    i = re.sub(r'\s+', ' ', i)
    
    i = i.strip()
    
    good_text_test.append(i)
test['comment_text'] = good_text_test

In [4]:
tfidf_train_1 = np.loadtxt('train_tfidf_data_1.txt')
tfidf_test_1 = np.loadtxt('test_tfidf_data_1.txt')

tfidf_train_2 = np.loadtxt('train_tfidf_data_2.txt')
tfidf_test_2 = np.loadtxt('test_tfidf_data_2.txt')

We see that we have 10 times more non toxic data than toxic data. Let us divide the non toxic comments into 5 pars and train 5 models with each part each.

In [19]:
train[train['sum_level']==0].shape

(143346, 9)

In [8]:
train[train['sum_level']>0].shape

(16225, 9)

In [20]:
143346/5

28669

In [30]:
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score

In [5]:
train_clean_index = train[train['sum_level']==0].index.values
train_toxic_index = train[train['sum_level']>0].index.values

In [31]:
x = np.concatenate((tfidf_train_1[train_clean_index[:28670],:], tfidf_train_1[train_toxic_index,:]), axis=0)
y = np.concatenate((np.array(train.iloc[train_clean_index[:28670],2:-1]), np.array(train.iloc[train_toxic_index,2:-1])), axis=0)

In [32]:
assert(x.shape[0]==y.shape[0])

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=2017)

In [42]:
class roc_callback(Callback):
    def __init__(self,training_data):
        self.x = training_data[0]
        self.y = training_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred, average='weighted')
        print('\rroc-auc: %s' % (str(round(roc,4)))+' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [102]:
#now we are ready for Deep learning. Wanted to start with boosting, but lets strt with DL. 
#we will start with a simple keras model

#considering text data let us start with a simple fully connected model


#starting keras model with tensorflow backend

inputs = Input(shape=(200,))

a = Dense(128,activation='tanh', input_shape=(None,200))(inputs)
a = Dropout(0.6)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.6)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.6)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.6)(a)
output= Dense(6, activation='sigmoid')(a)


model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[metrics.categorical_accuracy])

mck = ModelCheckpoint('sub_x1.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
estop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1, mode='auto')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               25728     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

In [44]:
model.fit(x_train,y_train,epochs=200, batch_size=80, verbose=1, shuffle=True, validation_split=0.3, callbacks=[mck,estop,roc_callback(training_data=(x_train,y_train))])

Train on 25141 samples, validate on 10775 samples
Epoch 1/200
24720/25141 [============================>.] - ETA: 0s - loss: 1.0824 - categorical_accuracy: 0.9435Epoch 00000: val_loss improved from inf to 1.05165, saving model to sub_x1.h5
roc-auc: 0.6644 

25141/25141 [==============================] - 5s - loss: 1.0831 - categorical_accuracy: 0.9442 - val_loss: 1.0517 - val_categorical_accuracy: 0.9786
Epoch 2/200
24720/25141 [============================>.] - ETA: 0s - loss: 1.0307 - categorical_accuracy: 0.9690Epoch 00001: val_loss improved from 1.05165 to 1.02609, saving model to sub_x1.h5
roc-auc: 0.7303 

25141/25141 [==============================] - 4s - loss: 1.0305 - categorical_accuracy: 0.9691 - val_loss: 1.0261 - val_categorical_accuracy: 0.9770
Epoch 3/200
24640/25141 [============================>.] - ETA: 0s - loss: 1.0165 - categorical_accuracy: 0.9683Epoch 00002: val_loss improved from 1.02609 to 1.01996, saving model to sub_x1.h5
roc-auc: 0.7013 

25141/25141 [=====

In [46]:
model.load_weights('sub_x1.h5')

y_pred = model.predict(x_test)
score = roc_auc_score(y_test,y_pred,average='weighted')
print(score)

0.763827757657


In [48]:
K.clear_session()

In [56]:
x = np.concatenate((tfidf_train_1[train_clean_index[28670:28670*2],:], tfidf_train_1[train_toxic_index,:]), axis=0)
y = np.concatenate((np.array(train.iloc[train_clean_index[28670:28670*2],2:-1]), np.array(train.iloc[train_toxic_index,2:-1])), axis=0)

In [57]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=1)

In [105]:
inputs = Input(shape=(200,))

a = Dense(128,activation='tanh', input_shape=(None,200))(inputs)
a = Dropout(0.6)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.6)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.6)(a)
output= Dense(6, activation='sigmoid')(a)


model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[metrics.categorical_accuracy])

mck = ModelCheckpoint('sub_x2.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
estop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1, mode='auto')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               25728     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

In [59]:
model.fit(x_train,y_train,epochs=200, batch_size=80, verbose=1, shuffle=True, validation_split=0.3, callbacks=[mck,estop,roc_callback(training_data=(x_train,y_train))])

Train on 25141 samples, validate on 10775 samples
Epoch 1/200
25040/25141 [============================>.] - ETA: 0s - loss: 1.0823 - categorical_accuracy: 0.9300Epoch 00000: val_loss improved from inf to 1.02349, saving model to sub_x2.h5
roc-auc: 0.6378 

25141/25141 [==============================] - 13s - loss: 1.0823 - categorical_accuracy: 0.9301 - val_loss: 1.0235 - val_categorical_accuracy: 0.9725
Epoch 2/200
25040/25141 [============================>.] - ETA: 0s - loss: 1.0267 - categorical_accuracy: 0.9623Epoch 00001: val_loss improved from 1.02349 to 1.00086, saving model to sub_x2.h5
roc-auc: 0.6812 

25141/25141 [==============================] - 12s - loss: 1.0262 - categorical_accuracy: 0.9624 - val_loss: 1.0009 - val_categorical_accuracy: 0.9658
Epoch 3/200
25040/25141 [============================>.] - ETA: 0s - loss: 1.0120 - categorical_accuracy: 0.9619Epoch 00002: val_loss improved from 1.00086 to 0.99424, saving model to sub_x2.h5
roc-auc: 0.7117 

25141/25141 [===

In [60]:
model.load_weights('sub_x2.h5')

y_pred = model.predict(x_test)
score = roc_auc_score(y_test,y_pred,average='weighted')
print(score)

0.772809472792


In [61]:
K.clear_session()

In [62]:
x = np.concatenate((tfidf_train_1[train_clean_index[28670*2:28670*3],:], tfidf_train_1[train_toxic_index,:]), axis=0)
y = np.concatenate((np.array(train.iloc[train_clean_index[28670*2:28670*3],2:-1]), np.array(train.iloc[train_toxic_index,2:-1])), axis=0)

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=634)

In [108]:
inputs = Input(shape=(200,))

a = Dense(128,activation='tanh', input_shape=(None,200))(inputs)
a = Dropout(0.5)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.5)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.5)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.5)(a)
output= Dense(6, activation='sigmoid')(a)


model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[metrics.categorical_accuracy])

mck = ModelCheckpoint('sub_x3.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
estop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1, mode='auto')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               25728     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

In [81]:
model.fit(x_train,y_train,epochs=200, batch_size=50, verbose=1, shuffle=True, validation_split=0.3, callbacks=[mck,estop,roc_callback(training_data=(x_train,y_train))])

Train on 25141 samples, validate on 10775 samples
Epoch 1/200
25000/25141 [============================>.] - ETA: 0s - loss: 1.0673 - categorical_accuracy: 0.9566Epoch 00000: val_loss improved from inf to 1.00154, saving model to sub_x3.h5
roc-auc: 0.6937 

25141/25141 [==============================] - 6s - loss: 1.0669 - categorical_accuracy: 0.9568 - val_loss: 1.0015 - val_categorical_accuracy: 0.9745
Epoch 2/200
24850/25141 [============================>.] - ETA: 0s - loss: 1.0191 - categorical_accuracy: 0.9643Epoch 00001: val_loss improved from 1.00154 to 0.98866, saving model to sub_x3.h5
roc-auc: 0.7108 

25141/25141 [==============================] - 7s - loss: 1.0210 - categorical_accuracy: 0.9644 - val_loss: 0.9887 - val_categorical_accuracy: 0.9793
Epoch 3/200
25100/25141 [============================>.] - ETA: 0s - loss: 1.0098 - categorical_accuracy: 0.9649Epoch 00002: val_loss improved from 0.98866 to 0.98690, saving model to sub_x3.h5
roc-auc: 0.7133 

25141/25141 [=====

In [82]:
model.load_weights('sub_x3.h5')

y_pred = model.predict(x_test)
score = roc_auc_score(y_test,y_pred,average='weighted')
print(score)

0.806991016792


In [83]:
K.clear_session()

In [84]:
x = np.concatenate((tfidf_train_1[train_clean_index[28670*3:28670*4],:], tfidf_train_1[train_toxic_index,:]), axis=0)
y = np.concatenate((np.array(train.iloc[train_clean_index[28670*3:28670*4],2:-1]), np.array(train.iloc[train_toxic_index,2:-1])), axis=0)

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=1264)

In [111]:
inputs = Input(shape=(200,))

a = Dense(128,activation='tanh', input_shape=(None,200))(inputs)
a = Dropout(0.6)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.6)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.6)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.6)(a)
output= Dense(6, activation='sigmoid')(a)


model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[metrics.categorical_accuracy])

mck = ModelCheckpoint('sub_x4.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
estop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1, mode='auto')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               25728     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

In [90]:
model.fit(x_train,y_train,epochs=200, batch_size=50, verbose=1, shuffle=True, validation_split=0.3, callbacks=[mck,estop,roc_callback(training_data=(x_train,y_train))])

Train on 25141 samples, validate on 10775 samples
Epoch 1/200
25000/25141 [============================>.] - ETA: 0s - loss: 1.0906 - categorical_accuracy: 0.9518Epoch 00000: val_loss improved from inf to 0.98967, saving model to sub_x4.h5
roc-auc: 0.6472 

25141/25141 [==============================] - 6s - loss: 1.0896 - categorical_accuracy: 0.9520 - val_loss: 0.9897 - val_categorical_accuracy: 0.9761
Epoch 2/200
25100/25141 [============================>.] - ETA: 0s - loss: 1.0451 - categorical_accuracy: 0.9683Epoch 00001: val_loss improved from 0.98967 to 0.96371, saving model to sub_x4.h5
roc-auc: 0.6804 

25141/25141 [==============================] - 6s - loss: 1.0448 - categorical_accuracy: 0.9683 - val_loss: 0.9637 - val_categorical_accuracy: 0.9736
Epoch 3/200
25000/25141 [============================>.] - ETA: 0s - loss: 1.0301 - categorical_accuracy: 0.9700Epoch 00002: val_loss did not improve
roc-auc: 0.741 

25141/25141 [==============================] - 6s - loss: 1.029

In [91]:
model.load_weights('sub_x4.h5')

y_pred = model.predict(x_test)
score = roc_auc_score(y_test,y_pred,average='weighted')
print(score)

0.816510195348


In [92]:
K.clear_session()

In [93]:
x = np.concatenate((tfidf_train_1[train_clean_index[28670*4:],:], tfidf_train_1[train_toxic_index,:]), axis=0)
y = np.concatenate((np.array(train.iloc[train_clean_index[28670*4:],2:-1]), np.array(train.iloc[train_toxic_index,2:-1])), axis=0)

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=2673)

In [114]:
inputs = Input(shape=(200,))

a = Dense(128,activation='tanh', input_shape=(None,200))(inputs)
a = Dropout(0.6)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.6)(a)
a = Dense(128,activation='tanh')(a)
a = Dropout(0.6)(a)
output= Dense(6, activation='sigmoid')(a)


model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[metrics.categorical_accuracy])

mck = ModelCheckpoint('sub_x5.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
estop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1, mode='auto')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               25728     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

In [99]:
model.fit(x_train,y_train,epochs=200, batch_size=50, verbose=1, shuffle=True, validation_split=0.3, callbacks=[mck,estop,roc_callback(training_data=(x_train,y_train))])

Train on 25138 samples, validate on 10774 samples
Epoch 1/200
25000/25138 [============================>.] - ETA: 0s - loss: 1.0810 - categorical_accuracy: 0.9548Epoch 00000: val_loss improved from inf to 1.02134, saving model to sub_x5.h5
roc-auc: 0.6555 

25138/25138 [==============================] - 6s - loss: 1.0817 - categorical_accuracy: 0.9547 - val_loss: 1.0213 - val_categorical_accuracy: 0.9686
Epoch 2/200
25000/25138 [============================>.] - ETA: 0s - loss: 1.0324 - categorical_accuracy: 0.9632Epoch 00001: val_loss improved from 1.02134 to 1.00176, saving model to sub_x5.h5
roc-auc: 0.6983 

25138/25138 [==============================] - 7s - loss: 1.0334 - categorical_accuracy: 0.9632 - val_loss: 1.0018 - val_categorical_accuracy: 0.9695
Epoch 3/200
25100/25138 [============================>.] - ETA: 0s - loss: 1.0215 - categorical_accuracy: 0.9652Epoch 00002: val_loss improved from 1.00176 to 0.99733, saving model to sub_x5.h5
roc-auc: 0.7153 

25138/25138 [=====

25138/25138 [==============================] - 20s - loss: 0.9936 - categorical_accuracy: 0.9774 - val_loss: 0.9881 - val_categorical_accuracy: 0.9774
Epoch 25/200
25000/25138 [============================>.] - ETA: 0s - loss: 0.9956 - categorical_accuracy: 0.9776Epoch 00024: val_loss did not improve
roc-auc: 0.7492 

25138/25138 [==============================] - 17s - loss: 0.9946 - categorical_accuracy: 0.9776 - val_loss: 0.9899 - val_categorical_accuracy: 0.9774
Epoch 26/200
25100/25138 [============================>.] - ETA: 0s - loss: 0.9914 - categorical_accuracy: 0.9777Epoch 00025: val_loss did not improve
roc-auc: 0.7822 

25138/25138 [==============================] - 17s - loss: 0.9910 - categorical_accuracy: 0.9778 - val_loss: 0.9886 - val_categorical_accuracy: 0.9774
Epoch 27/200
25000/25138 [============================>.] - ETA: 0s - loss: 0.9959 - categorical_accuracy: 0.9774Epoch 00026: val_loss did not improve
roc-auc: 0.7801 

25138/25138 [===========================

In [100]:
model.load_weights('sub_x5.h5')

y_pred = model.predict(x_test)
score = roc_auc_score(y_test,y_pred,average='weighted')
print(score)

0.789489397956


In [113]:
K.clear_session()

In [103]:
model.load_weights('sub_x1.h5')
y_pred_1 = model.predict(tfidf_test_1)

In [106]:
model.load_weights('sub_x2.h5')
y_pred_2 = model.predict(tfidf_test_1)

In [109]:
model.load_weights('sub_x3.h5')
y_pred_3 = model.predict(tfidf_test_1)

In [112]:
model.load_weights('sub_x4.h5')
y_pred_4 = model.predict(tfidf_test_1)

In [115]:
model.load_weights('sub_x5.h5')
y_pred_5 = model.predict(tfidf_test_1)

In [122]:
bagging_pred = y_pred_1+y_pred_2+y_pred_3+y_pred_4+y_pred_5
bagging_pred = bagging_pred/5

In [124]:
sub = pd.DataFrame(test['id'])

for i in range(len(levels)):
    sub[levels[i]]=bagging_pred[:,i]
sub.to_csv('bagging_pred.csv', index=False)

In [116]:
sub = pd.DataFrame(test['id'])

for i in range(len(levels)):
    sub[levels[i]]=y_pred_1[:,i]
sub.to_csv('pred_1.csv', index=False)

In [117]:
sub = pd.DataFrame(test['id'])

for i in range(len(levels)):
    sub[levels[i]]=y_pred_2[:,i]
sub.to_csv('pred_2.csv', index=False)

In [118]:
sub = pd.DataFrame(test['id'])

for i in range(len(levels)):
    sub[levels[i]]=y_pred_3[:,i]
sub.to_csv('pred_3.csv', index=False)

In [119]:
sub = pd.DataFrame(test['id'])

for i in range(len(levels)):
    sub[levels[i]]=y_pred_4[:,i]
sub.to_csv('pred_4.csv', index=False)

#This is the best till now

In [120]:
sub = pd.DataFrame(test['id'])

for i in range(len(levels)):
    sub[levels[i]]=y_pred_5[:,i]
sub.to_csv('pred_5.csv', index=False)